In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")


### Go through the lesson commenting code, adding docstrings, and adding markdown to support what is happening.



#### Acquire

Upon acquiring the df from the text file, it was found to not have any headers or column names and the first row was set to be the header. The column names were found from the database curriculum_logs and were provided as part of reading in the file. The time column was not included during acquisition.

In [3]:
def acquire(file_name, column_names):
    '''
    This function takes in a file to read and the column names to be added to the columns. 
    It returns the file as a pandas dataframe and does not include column 1.
    '''
    return pd.read_csv(file_name, sep="\s", header=None, names=column_names, usecols=[0, 2, 3, 4, 5])

In [4]:
file_name = 'anonymized-curriculum-access-07-2021.txt'
colnames = ['date', 'endpoint', 'user_id', 'cohort_id', 'source_ip']

# call the function to acquire the data
df = acquire(file_name, colnames)

In [5]:
df.head()

,date,endpoint,user_id,cohort_id,source_ip
0,2018-01-26,/,1,8.0,97.105.19.61
1,2018-01-26,java-ii,1,8.0,97.105.19.61
2,2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018810 entries, 0 to 1018809
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   date       1018810 non-null  object 
 1   endpoint   1018809 non-null  object 
 2   user_id    1018810 non-null  int64  
 3   cohort_id  965313 non-null   float64
 4   source_ip  1018810 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 38.9+ MB


#### Prepare

Since the date column has an object dtype, this will be converted to a datetime object before we can begin time series analysis.

In [8]:
def prep(df, user):
    '''
    This function takes in a df and a single user. 
    It converts the date column to a datetime object and sets that as the index.
    It also resamples daily counts for number of page views.
    '''
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages = df['endpoint'].resample('d').count()
    return pages


def compute_pct_b(pages, span, weight, user):
    '''
    This function takes in the pages, the number of days, the weight to calculate the standard deviation and the user.
    It calculates the mid, upper and lower bollinger bands and concatenates the upper and lower bands.
    It then creates a new dataframe with the number of page view, midband, upper and lower bands.
    It calculate the %b and adds a user id column to the df and then returns the df.
    '''
    midband = pages.ewm(span=span).mean()
    stdev = pages.ewm(span=span).std()
    ub = midband + stdev * weight
    lb = midband - stdev * weight
    bb = pd.concat([ub, lb], axis=1)
    my_df = pd.concat([pages, midband, bb], axis=1)
    my_df.columns = ['pages', 'midband', 'ub', 'lb']
    my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

def plt_bands(my_df, user):
    '''
    This function takes in a df and a user id.
    It plots the upper, lower and mid bollinger bands against the number of page views.
    '''
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()
    
def find_anomalies(df, user, span, weight):
    '''
    This function takes in a df, user id, number of days as span and the weight to compute %b. 
    It then prepares the df by calling the prep function.
    It also creates a new dataframe by calling the compute pct b function.
    It returns a subset of the df where %b is greater than 1.
    '''
    pages = prep(df, user)
    my_df = compute_pct_b(pages, span, weight, user)
    return my_df[my_df.pct_b > 1]

In [11]:
# test on a single user

user = 68
span = 30
weight = 3

# initialize empty df
anomalies = pd.DataFrame()

# call function to create a new df with anomalies for the user 
user_df = find_anomalies(df, user, span, weight)

# concatenate the above df created with the empty anomalies df
anomalies = pd.concat([anomalies, user_df], axis=0)

In [12]:
# call the df
anomalies

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2018-05-04,67,7.035731,60.259707,-46.188246,1.063320,68
2018-06-25,50,4.814414,45.293820,-35.664992,1.058131,68
2018-08-20,17,1.240614,15.592025,-13.110798,1.049054,68
2018-10-16,13,0.898680,10.970734,-9.173375,1.100737,68
2018-11-15,27,1.863468,22.364497,-18.637561,1.113055,68
2019-01-30,21,1.407805,17.354013,-14.538403,1.114321,68
2019-03-25,24,1.622558,19.932129,-16.687012,1.111086,68
2019-06-27,9,0.587115,7.439000,-6.264770,1.113910,68
2019-07-10,14,1.554444,13.962988,-10.854099,1.001491,68


In [13]:
user_df

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2018-05-04,67,7.035731,60.259707,-46.188246,1.063320,68
2018-06-25,50,4.814414,45.293820,-35.664992,1.058131,68
2018-08-20,17,1.240614,15.592025,-13.110798,1.049054,68
2018-10-16,13,0.898680,10.970734,-9.173375,1.100737,68
2018-11-15,27,1.863468,22.364497,-18.637561,1.113055,68
2019-01-30,21,1.407805,17.354013,-14.538403,1.114321,68
2019-03-25,24,1.622558,19.932129,-16.687012,1.111086,68
2019-06-27,9,0.587115,7.439000,-6.264770,1.113910,68
2019-07-10,14,1.554444,13.962988,-10.854099,1.001491,68


In [14]:
# loop through all users to find anomalies for each
span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [15]:
anomalies

,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2019-01-29,44,3.623334,42.081344,-34.834677,1.024945,1
2019-07-08,101,8.248768,96.507472,-80.009936,1.025451,1
2018-10-15,9,0.581421,8.457414,-7.294571,1.034446,3
2019-01-09,4,0.262470,3.802115,-3.277175,1.027953,3
2019-04-06,2,0.129825,1.889712,-1.630061,1.031334,3
...,...,...,...,...,...,...
2021-06-02,72,10.456225,71.377304,-50.464854,1.005111,925
2021-06-29,32,3.881578,31.571606,-23.808449,1.007736,929
2021-06-11,44,5.582783,43.858655,-32.693089,1.001846,961


- The anomalies df provides the anomalies found for each user in the number of page views. If the number of page views is very high we would be more interested in that vs very low number of page views.

In [17]:
# look at value counts for pages in anomalies without sorting the counts
anomalies.pages.value_counts(sort = False)

1      56
2      40
3      85
4      87
5      62
       ..
179     1
192     1
198     1
272     1
343     1
Name: pages, Length: 105, dtype: int64

- The highest number of page view is 343 so let's investigate that.

In [19]:
# look at a subset of anomalies where the number of page views is equal to 343
anomalies[anomalies.pages == 343]


,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2021-06-21,343,22.370564,322.155049,-277.413921,1.034767,804


- It looks like user_id 804 had this extremely high number of page views on 6/21.
- Let's investigate if this user also had any other days with such high number of page views.

In [20]:
# pull a subset of data where user id is 804
anomalies[anomalies.user_id == 804]


,pages,midband,ub,lb,pct_b,user_id
date,,,,,,
2021-01-20,74,8.832557,73.838286,-56.173171,1.001244,804
2021-06-21,343,22.370564,322.155049,-277.413921,1.034767,804


- This same user also viewed 74 pages on 1/20.
- We can investigate the pages this user viewed as well as their cohort and if they accessed the pages from the same ip.

In [23]:
# pull subset of df for only user 804
df[df.user_id == 804]

,date,endpoint,user_id,cohort_id,source_ip
719859,2020-11-03,javascript-i,804,132.0,69.91.64.132
720925,2020-11-04,javascript-i,804,132.0,69.91.64.132
721318,2020-11-04,javascript-i/javascript-with-html,804,132.0,69.91.64.132
721321,2020-11-04,javascript-i,804,132.0,69.91.64.132
721324,2020-11-04,javascript-i/introduction/primitive-types,804,132.0,69.91.64.132
...,...,...,...,...,...
987643,2021-06-21,appendix/further-reading/pagination,804,132.0,66.69.1.31
987644,2021-06-21,appendix/further-reading/authorization,804,132.0,66.69.1.31
987645,2021-06-21,appendix/further-reading/security-use-cases,804,132.0,66.69.1.31
987646,2021-06-21,appendix/further-reading/spring,804,132.0,66.69.1.31


- It looks like this user has some different ip's that they are using to access the curriculum. 
- This user seems to be a web dev student based on the endpoints.

In [24]:
# check the different ip's this user used
df[df.user_id == 804].source_ip.value_counts()

66.69.1.31        937
69.91.64.132      645
156.146.43.65      13
181.214.227.47      6
154.3.250.108       3
172.56.15.73        2
172.56.7.59         1
181.214.227.23      1
154.21.28.158       1
Name: source_ip, dtype: int64

- This user seems to be using quite a few ip's to access the curriculum. However, it is clear that one ip address is used most often.